In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

## This is the css selector for basic and advanced stats table on each page for each team.

In [2]:
# IDs of tables to scrape
table_ids = [
    'VEG_skaters', 'VEG_adv_ALLAll',
    'LAK_skaters', 'LAK_adv_ALLAll',
    'TBL_skaters', 'TBL_adv_ALLAll',
    'NYR_skaters', 'NYR_adv_ALLAll',
    'SEA_skaters', 'SEA_adv_ALLAll',
    'ANA_skaters', 'ANA_adv_ALLAll',
    'SJS_skaters', 'SJS_adv_ALLAll',
    'CBJ_skaters', 'CBJ_adv_ALLAll',
    'CAR_skaters', 'CAR_adv_ALLAll',
    'CHI_skaters', 'CHI_adv_ALLAll',
    'COL_skaters', 'COL_adv_ALLAll',
    'VAN_skaters', 'VAN_adv_ALLAll',
    'EDM_skaters', 'EDM_adv_ALLAll',
    'TOR_skaters', 'TOR_adv_ALLAll',
    'MTL_skaters', 'MTL_adv_ALLAll',
    'BOS_skaters', 'BOS_adv_ALLAll',
    'WSH_skaters', 'WSH_adv_ALLAll',
    'OTT_skaters', 'OTT_adv_ALLAll',
    'BUF_skaters', 'BUF_adv_ALLAll',
    'CGY_skaters', 'CGY_adv_ALLAll',
    'MIN_skaters', 'MIN_adv_ALLAll',
    'FLA_skaters', 'FLA_adv_ALLAll',
    'NYI_skaters', 'NYI_adv_ALLAll',
    'DAL_skaters', 'DAL_adv_ALLAll',
    'NSH_skaters', 'NSH_adv_ALLAll',
    'DET_skaters', 'DET_adv_ALLAll',
    'ARI_skaters', 'ARI_adv_ALLAll',
    'PIT_skaters', 'PITT_adv_ALLAll',
    'NJD_skaters', 'NJD_adv_ALLAll',
    'PHI_skaters', 'PHI_adv_ALLAll',
    'STL_skaters', 'STL_adv_ALLAll',
    'WPG_skaters', 'WPG_adv_ALLAll',
    'ATL_skaters', 'ATL_adv_ALLAll',  # Atlanta Thrashers (now Winnipeg Jets)
    'PHX_skaters', 'PHX_adv_ALLAll',  # Phoenix Coyotes (now Arizona Coyotes)
]

#Team names to add it later
team_names = {
    'VEG': 'Vegas Golden Knights',
    'LAK': 'Los Angeles Kings',
    'TBL': 'Tampa Bay Lightning',
    'NYR': 'New York Rangers',
    'SEA': 'Seattle Kraken',
    'ANA': 'Anaheim Ducks',
    'SJS': 'San Jose Sharks',
    'CBJ': 'Columbus Blue Jackets',
    'CAR': 'Carolina Hurricanes',
    'CHI': 'Chicago Blackhawks',
    'COL': 'Colorado Avalanche',
    'VAN': 'Vancouver Canucks',
    'EDM': 'Edmonton Oilers',
    'TOR': 'Toronto Maple Leafs',
    'MTL': 'Montreal Canadiens',
    'BOS': 'Boston Bruins',
    'WSH': 'Washington Capitals',
    'OTT': 'Ottawa Senators',
    'BUF': 'Buffalo Sabres',
    'CGY': 'Calgary Flames',
    'MIN': 'Minnesota Wild',
    'FLA': 'Florida Panthers',
    'NYI': 'New York Islanders',
    'DAL': 'Dallas Stars',
    'NSH': 'Nashville Predators',
    'DET': 'Detroit Red Wings',
    'ARI': 'Arizona Coyotes',
    'PIT': 'Pittsburgh Penguins',
    'NJD': 'New Jersey Devils',
    'PHI': 'Philadelphia Flyers',
    'STL': 'St. Louis Blues',
    'WPG': 'Winnipeg Jets',
    'ATL': 'Atlanta Thrashers',  # Adding Atlanta Thrashers (now Winnipeg Jets)
    'PHX': 'Phoenix Coyotes',    # Adding Phoenix Coyotes (now Arizona Coyotes)
}


## This block is used to scrape the player performance for each team's players in each game till 2010

In [5]:
#initialising driver
driver = webdriver.Chrome()
try:
    #main schedule page url
    url = 'https://www.hockey-reference.com/leagues/NHL_2013_games.html'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #parsing the date section to go on individual web page of each match and storing it in a list
    links = [a['href'] for a in soup.select('table#games th[data-stat="date_game"] > a')]

    #lists for extracted DFs
    basic_stats = []
    adv_stats = []
    for link in links:
        #splitting date to add in later 
        date_str = link.split('/')[-1][:8]  
        date = datetime.strptime(date_str, '%Y%m%d')
        if 'http' not in link:
            link = 'https://www.hockey-reference.com' + link

        #using selenium to traverse on each page
        driver.get(link)
        time.sleep(2)  
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Finding basic and advanced stats according to the table id
        for table_id in table_ids:
            table = soup.find('table', {'id': table_id})
            if table:
                df = pd.read_html(str(table))[0]

                #the columns are multilevel indexed
                # Getting the second level index for better manipulation 
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.droplevel(0)
                
                #adding team name and date to find trends in the future
                acronym = table_id.split('_')[0] 
                df['team'] = team_names.get(acronym, 'Unknown') 
                df['date'] = date
                if 'skaters' in table_id:
                    basic_stats.append(df)
                elif 'adv_ALLAll' in table_id:
                    adv_stats.append(df)

    # Close WebDriver
    driver.quit()
except Exception as e:
    print(e)

Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000104567f48 chromedriver + 4226888
1   chromedriver                        0x00000001045604f4 chromedriver + 4195572
2   chromedriver                        0x00000001041a4d68 chromedriver + 281960
3   chromedriver                        0x000000010417eef8 chromedriver + 126712
4   chromedriver                        0x0000000104203e88 chromedriver + 671368
5   chromedriver                        0x000000010421645c chromedriver + 746588
6   chromedriver                        0x00000001041d3f1c chromedriver + 474908
7   chromedriver                        0x00000001041d4ef4 chromedriver + 478964
8   chromedriver                        0x000000010452959c chromedriver + 3970460
9   chromedriver                        0x000000010452d6f0 chromedriver + 3987184
10  chromedriver                        0

In [30]:
#concatenating all dataframes to a single dataframe
basic_pp = pd.concat(basic_stats, ignore_index=True)
adv_pp = pd.concat(adv_stats, ignore_index=True)

In [31]:
#merging the datframes
player_performance = pd.merge(basic_pp, adv_pp, on=['Player', 'date', 'team'], suffixes=('_basic', '_adv'))

In [32]:
#re-ordering the dataframe for consistent analysis
cols = list(player_performance.columns)
cols = [col for col in cols if col not in ['team', 'date']] + ['team', 'date']

player_performance = player_performance[cols]

In [34]:
player_performance

,Rk,Player,G,A,PTS,+/-,PIM,EV,PP,SH,...,SAT‑A,CF%,CRel%,ZSO,ZSD,oZS%,HIT,BLK,team,date
0,1.0,Bryan Bickell,1.0,0.0,1,0.0,0,0.0,1.0,0.0,...,14,56.2,56.2,4.0,4.0,50.0,2,0.0,Chicago Blackhawks,2010-10-07
1,2.0,Dave Bolland,0.0,0.0,0,-1.0,0,0.0,0.0,0.0,...,15,37.5,37.5,6.0,10.0,37.5,0,0.0,Chicago Blackhawks,2010-10-07
2,3.0,Troy Brouwer,0.0,1.0,1,0.0,0,0.0,0.0,0.0,...,14,51.7,51.7,3.0,5.0,37.5,1,0.0,Chicago Blackhawks,2010-10-07
3,4.0,Jake Dowell,0.0,0.0,0,-1.0,0,0.0,0.0,0.0,...,7,50.0,50.0,3.0,2.0,60.0,1,0.0,Chicago Blackhawks,2010-10-07
4,5.0,Jordan Hendry,0.0,0.0,0,0.0,0,0.0,0.0,0.0,...,4,55.6,55.6,4.0,0.0,100.0,1,2.0,Chicago Blackhawks,2010-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42044,15.0,Mattias Tedenby,0.0,0.0,0,0.0,0,0.0,0.0,0.0,...,6,73.9,25.3,5.0,6.0,45.5,0,0.0,New Jersey Devils,2011-04-10
42045,16.0,Alexander Urbom,1.0,0.0,1,1.0,0,1.0,0.0,0.0,...,14,48.1,-9.0,6.0,2.0,75.0,1,1.0,New Jersey Devils,2011-04-10
42046,17.0,Travis Zajac,0.0,0.0,0,-1.0,0,0.0,0.0,0.0,...,17,45.2,-13.9,2.0,6.0,25.0,0,0.0,New Jersey Devils,2011-04-10
42047,18.0,Vladimir Zharkov,1.0,0.0,1,1.0,0,1.0,0.0,0.0,...,7,65.0,13.1,2.0,3.0,40.0,0,0.0,New Jersey Devils,2011-04-10


In [33]:
#exporting to csv
player_performance.to_csv("PP_2011.csv")